## Two phase flow.
### pezoflow equation.

$$\frac{\partial S_w m}{\partial t} + \frac{\partial}{\partial x}(W_w)=0$$
$$\frac{\partial S_o m}{\partial t} + \frac{\partial}{\partial x}(W_o)=0$$
$$W_w=-\frac{k_{rw} k}{\mu_w}\frac{\partial p}{\partial x}$$
$$W_o=-\frac{k_{ro} k}{\mu_o}\frac{\partial p}{\partial x}$$
$$S_o+S_w=1$$
$$m(p)=m_0+\beta_r*(p-p_0)$$

$$x\in[0,L]$$
$$t\in[0T]$$

*Initial and boundary conditions*
$$p(x, t=0)=p_0$$
$$S_w(x, t=0)=S_{w0}$$
$$W_w(x=0, t)=\frac{Q}{S}$$
$$S_w(x=0, t)=1$$
$$p(x=1, t)=p_0$$

*non dimensions parameters*
$$\overline{p}=\frac{p}{p_0}, \overline{t}=\frac{t}{T}, \overline{x}=\frac{x}{L}$$

1.
$$m\frac{\partial S_w}{\partial t}+S_w\frac{\partial m}{\partial t} + \frac{\partial}{\partial x}(W_w)=0$$
$$m\frac{\partial S_o}{\partial t}+S_o\frac{\partial m}{\partial t} + \frac{\partial}{\partial x}(W_o)=0$$



2.
$$m\frac{\partial S_w}{\partial t}+S_w\beta_r\frac{\partial p}{\partial t} + \frac{\partial}{\partial x}(W_w)=0$$
$$m\frac{\partial (1-S_w)}{\partial t}+(1-S_w)\beta_r\frac{\partial p}{\partial t} + \frac{\partial}{\partial x}(W_o)=0$$



3. summarize
$$\beta_r\frac{\partial p}{\partial t} + \frac{\partial}{\partial x}(W_o+W_w)=0$$
$$\beta_r\frac{\partial p}{\partial t} - \frac{\partial}{\partial x}(k(\frac{k_{rw}}{\mu_w}+\frac{k_{ro}}{\mu_o})\frac{\partial p}{\partial x})=0$$

final
$$\beta_r\frac{\partial p}{\partial t} - \frac{\partial}{\partial x}(k(\frac{k_{rw}}{\mu_w}+\frac{k_{ro}}{\mu_o})\frac{\partial p}{\partial x})=0$$
$$m\frac{\partial S_w}{\partial t}+S_w\beta_r\frac{\partial p}{\partial t} + \frac{\partial}{\partial x}(W_w)=0$$

*non dimensional*
$$\frac{\beta_r p_0}{T}\frac{\partial \overline{p}}{\partial \overline{t}} - \frac{1}{L}\frac{\partial}{\partial \overline{x}}(k(\frac{k_{rw}}{\mu_w}+\frac{k_{ro}}{\mu_o})\frac{p_0}{L}\frac{\partial \overline{p}}{\partial \overline{x}})=0$$
$$\frac{m}{T}\frac{\partial S_w}{\partial \overline{t}}+\frac{S_w\beta_r p_0}{T}\frac{\partial \overline{p}}{\partial \overline{t}} + \frac{1}{L}\frac{\partial}{\partial \overline{x}}(k\frac{k_{rw}}{\mu_w}\frac{p_0}{L}\frac{\partial \overline{p}}{\partial \overline{x}})=0$$

*conditions*
$$\frac{k p_0}{L}\frac{k_{rw}}{\mu_w}\frac{\partial \overline{p}}{\partial \overline{x}} = \frac{Q}{S}$$

*non dimensional final*
$$\beta_r\frac{\partial \overline{p}}{\partial \overline{t}} - \frac{T}{ L^2}\frac{\partial}{\partial \overline{x}}(k(\frac{k_{rw}}{\mu_w}+\frac{k_{ro}}{\mu_o})\frac{\partial \overline{p}}{\partial \overline{x}})=0$$
$$\frac{\partial S_w}{\partial \overline{t}}+\frac{S_w\beta_r p_0}{m}\frac{\partial \overline{p}}{\partial \overline{t}} + \frac{p_0 T}{m L^2}\frac{\partial}{\partial \overline{x}}(k\frac{k_{rw}}{\mu_w}\frac{\partial \overline{p}}{\partial \overline{x}})=0$$

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve, lsqr
from scipy.optimize import newton_krylov, minimize, fsolve
from scipy.interpolate import interp1d
from sol_plot import solution_plot


def expit(x):
    return 1 / (1 + np.exp(-x))

k_water = lambda x: 0.6*(x)**2
k_oil = lambda x: 0.2*(x-1)**2

dk_water = lambda x: 1.2*x
dk_oil = lambda x: 0.4*(x-1)

plt.plot(np.linspace(0,1,100), k_water(np.linspace(0,1,100)))
plt.plot(np.linspace(0,1,100), k_oil(np.linspace(0,1,100)))

ModuleNotFoundError: No module named 'sol_plot'

final
$$\beta_r\frac{\partial p}{\partial t} - \frac{\partial}{\partial x}(k(\frac{k_{rw}}{\mu_w}+\frac{k_{ro}}{\mu_o})\frac{\partial p}{\partial x})=0$$
$$m\frac{\partial S_w}{\partial t}+S_w\beta_r\frac{\partial p}{\partial t} + \frac{\partial}{\partial x}(W_w)=0$$

In [27]:
class TwoPhase():
    def __init__(self,
                 mu_h,
                 mu_o,
                 glad,
                 G,
                 mu_water,
                 k,
                 Q,
                 m,
                 beta_r,
                 t_0,
                 T,
                 nt,
                 tt,
                 pow_n,
                 alfa_k,
                 x_0,
                 L,
                 nx,
                 p_0,
                 s_0,
                 s_left,
                 s_right,
                 mu_type) -> None:
        self.mu_h = mu_h
        self.mu_o = mu_o
        self.glad = glad
        self.G = G
        self.mu_water = mu_water
        self.k = k
        self.Q = Q
        self.m = m
        self.beta_r = beta_r
        self.t_0 = t_0
        self.T = T
        self.nt = nt
        self.tt = tt
        self.pow_n = pow_n
        self.x_0 = x_0
        self.L = L
        self.nx = nx
        self.p_0 = p_0
        self.s_0 = s_0
        self.s_left = s_left
        self.s_right = s_right
        self.alfa_k = alfa_k

        self.time = np.linspace(t_0, T, int(nt), dtype=np.float64)
        self.dt = self.time[1]-self.time[0]
        self.tt = self.time.flat[np.abs(self.time - tt).argmin()]
        self.x = np.linspace(0, L, nx, dtype=np.float64)
        # self.x_middle = np.linspace((self.x[1]-self.x[0])/2, self.x[-1]-(self.x[-1]-self.x[-2])/2, self.nx-1, dtype=np.float64)
        self.dx = self.x[1]-self.x[0]
        self.mu_oil_arr = []
        self.grad_p = []
        self.mu_stop = np.zeros_like(self.x)
        if mu_type not in ["mu_stop", "mu_run"]:
            raise NameError
        
        self.mu_type = mu_type

    def mu_oil(self, grad, mu_stop):
        grad = abs(grad)
        mu = (self.mu_h-self.mu_o) * expit (self.glad * (-grad + self.G)) + self.mu_o
        if self.mu_type=='mu_stop':
            res = np.where(mu<=1.05*self.mu_o, 1, 0)
            res = np.where(mu_stop+res>=1, 1, 0)
            result = np.where(res==1, self.mu_o, mu)
        else:
            result = mu
        return result

    def rate(self, t):
        n = self.pow_n
        a = self.Q*(self.T-self.t_0) / (self.tt**(n+1)/(n+1) + self.tt**n*(self.T-self.tt))
        return np.where(t<self.tt,a*t**(n),a*self.tt**(n))

    def lam_w(self, s):
        return k_water(s) / self.mu_water

    def lam_o(self, s, grad, mu):
        return k_oil(s) / self.mu_oil(grad, mu)

    def beta(self, s, grad, mu):
        return (self.lam_o(s, grad, mu) + self.lam_w(s)) * self.k

    def solution_init(self):
        p = np.zeros((self.nt, self.nx), dtype=np.float64)
        p[0, :] = self.p_0

        s = np.zeros((self.nt, self.nx), dtype=np.float64)
        s[0,:] = self.s_0
        s[:,0] = self.s_left
        return p, s

    def residual(self, u_new, u_old, s_old, t_step, dt, dx):
        """Выражение невязки для метода Ньютона-Крылова."""
        N = len(u_new)
        res = np.zeros(N, dtype=np.float64)
        grad = np.gradient(u_new)
        # Ss_weno = weno4(self.x_middle, self.x, s_old)
        # k_left = (self.beta(s_old[:-2], grad[:-2], grad_old[:-2], mu[:-2])+self.beta(s_old[1:-1], grad[1:-1], grad_old[1:-1], mu[1:-1])) / 2
        # k_right = (self.beta(s_old[2:], grad[2:], grad_old[2:], mu[2:])+self.beta(s_old[1:-1], grad[1:-1], grad_old[1:-1], mu[1:-1])) / 2
        k_left = self.beta(s_old[1:-1], grad[1:-1], self.mu_stop[1:-1])
        k_right = self.beta(s_old[2:], grad[2:], self.mu_stop[2:])
        res[1:-1] = self.beta_r*(u_new[1:-1] - u_old[1:-1]) / dt - \
                    (k_right * (u_new[2:] - u_new[1:-1]) / dx**2 - k_left * (u_new[1:-1] - u_new[:-2]) / dx**2)

        # Граничные условия
        res[0] = (u_new[1] - u_new[0]) / dx * k_water(s_old[0]) / self.mu_water*self.k + self.rate(t_step)
        res[-1] = u_new[-1] - self.p_0

        return res

    # Метод Ньютона-Крылова для решения нелинейной системы
    def solve_nonlinear(self, u_old, s_old, t_step, dt, dx):
        """Решение системы уравнений с использованием метода Ньютона-Крылова."""
        u_new_guess = np.copy(u_old)  # начальное предположение
        u_new = newton_krylov(lambda u_new: self.residual(u_new, u_old, s_old, t_step, dt, dx), u_new_guess, x_rtol=1e-4)
        return u_new

    # Параметры фракционного потока
    def fw(self, s, p):
        """ Функция фракционного потока воды """
        grad = np.gradient(p, self.dx)

        return self.lam_w(s) * self.k * grad

    def exact(self, t):
        Swi0 = self.s_0

        def f(sat):
            return k_water(sat)/(k_water(sat)+self.mu_water/self.mu_o*k_oil(sat))

        def df(sat):
            return (dk_water(sat)*(k_water(sat)+self.mu_water/self.mu_o*k_oil(sat))-k_water(sat)*(dk_water(sat)+self.mu_water/self.mu_o*dk_oil(sat)))/(k_water(sat)+self.mu_water/self.mu_o*k_oil(sat))**2

        def opt(sat):
            return f(sat)-f(Swi0)-df(sat)*(sat-Swi0)

        Sc = fsolve(opt, 0.4)
        Xc = self.Q/self.m*(f(Sc)-f(Swi0))/(Sc-Swi0)*t
        Sx1 = np.zeros(200)
        X1 = np.zeros(200)
        Sx2 = np.zeros(200)
        X2 = np.zeros(200)
        Sx1[0] = 1
        Sx2[0] = Swi0
        X2[0] = Xc
        for i in range(1,200):
            Sx1[i] = Sx1[i-1]-(1-Sc)/199 
            Sx2[i] = Swi0
            X2[i] = X2[i-1] + (self.L+1e-3-Xc)/199
        for i in range(1,200):
            X1[i] = self.Q/self.m * df(Sx1[i]) * t
        X3 = np.array([Xc[0], Xc[0]])
        Sx3 = np.array([Sc, Swi0])
        return np.concatenate((X1,X3,X2)), np.concatenate((Sx1,Sx3,Sx2))

    def solve(self):
        p, s = self.solution_init()
        s_buckley = np.zeros_like(s)
        s_buckley[0,:] = s[0,:]
        mu_oil_init = np.zeros(self.nx, dtype=np.float64)+self.mu_h
        self.mu_oil_arr.append(mu_oil_init)
        self.grad_p.append(np.gradient(p[0,:], self.dx))
        # Решение системы с использованием Ньютона-Крылова
        for t, t_step in enumerate(self.time[:-1]):
            # решение уравнения для давления
            p_old = p[t, :].copy()
            s_old = s[t, :].copy()
            p_new = self.solve_nonlinear(p_old, s_old, t_step, self.dt, self.dx)
            p[t+1, :] = p_new.copy()
            grad_new = np.gradient(p_new, self.dx)
            # решение уравнения для насыщенности
            fw_val = self.fw(s_old, p_new)
            s_new = np.zeros_like(s[t,:], dtype=np.float64)
            s_new[1:-1] = (s_old[1:-1] + self.dt/(self.m*self.dx) * (fw_val[1:-1] - fw_val[:-2])) - s_old[1:-1]*self.beta_r/self.m*(p_new[1:-1]-p_old[1:-1])
            s_new[0] = self.s_left
            s_new[-1] = s_new[-2]
            s[t+1,:] = s_new.copy()
            
            if t>=0:
                mu_oil_new = self.mu_oil(grad_new, self.mu_stop)
                res = np.where(mu_oil_new==self.mu_o, 1, 0)
                self.mu_stop = np.where(self.mu_stop+res>=1, 1, 0)
            self.mu_oil_arr.append(mu_oil_new)
            self.grad_p.append(grad_new)
            x_b, s_b = self.exact(t_step)
            func = interp1d(x_b, s_b, kind='nearest')
            s_b = func(self.x)
            s_buckley[t+1, :] = s_b
        self.mu_oil_arr = np.array(self.mu_oil_arr)
        self.grad_p = np.array(self.grad_p)
        return p, s, x_b, s_buckley

рассмотрим влияение следующих параметров на решение:

1. $$\mu_h \text{-------верхняя полка на графике вязкости}$$
2. $$glad \text{--------скорость изменения вязкости}$$
3) $$G \text{--------предельный градиент}$$
4) $$Q \text{------- расход}$$
5) $$tt \text{------момент времени до которого расход растет, после - постоянный}$$
6) $$s_0 \text{------начальная водонасыщенность, объем неподвижной воды в образце = 0}$$
7) $$\beta_r \text{------сжимаемость скелета}$$
7) Закон изменения вязкости,
    mu_stop - вязкость при достижении нижней полки не может увеличиваться
    mu_run -  вязкость может как увеличиваться, так и уменьшаться

In [ ]:
solver = TwoPhase(mu_h = 2e-3,
                mu_o = 2e-3,
                glad = 0.00002,
                G = 0.4e7,
                mu_water = 8e-4,
                k = 10e-15,
                Q = 0.4/86400,
                m = 0.2,
                beta_r = 0,
                t_0 = 0,
                T = 10*86400,
                nt = 150*5,
                tt = 0.001*864000,
                pow_n = 2,
                alfa_k=1e-9,
                x_0 = 0,
                L = 100,
                nx = 500,
                p_0 = 1e+6,
                s_0 = 0.,
                s_left = 1,
                s_right = 1,
                mu_type = "mu_stop")

p, s, x_b, s_b = solver.solve()

## Верификация численного решения с решением уравнения БЛ

In [ ]:
G = -np.ones_like(solver.grad_p)*solver.G

solution_plot([[solver.x, p],
               [solver.x, s],
               [solver.x, s_b],
               [solver.x, solver.mu_oil_arr],
               [solver.x, solver.grad_p],
               [solver.x, G]],
               titles=("Pressure (p) vs x", "Saturation (s) vs x", "mu_oil", "grad(p)"),
               idx=[[1,1], [1,2], [1,2], [2,1], [2,2], [2,2]],
               nt=solver.nt,
               height=800,
               width=1200)


## Влияние сжмаемости скелета при линейном течении

1) при увеличении сжимаемости скелета, скорость фронта насыщенности становится меньше.

In [ ]:
solver = TwoPhase(mu_h = 2e-3,
                mu_o = 2e-3,
                glad = 0.00002,
                G = 0.4e7,
                mu_water = 8e-4,
                k = 10e-15,
                Q = 0.4/86400,
                m = 0.2,
                beta_r = 1e-9,
                t_0 = 0,
                T = 10*86400,
                nt = 150*5,
                tt = 0.001*864000,
                pow_n = 2,
                alfa_k=1e-9,
                x_0 = 0,
                L = 100,
                nx = 500,
                p_0 = 1e+6,
                s_0 = 0.,
                s_left = 1,
                s_right = 1,
                mu_type = "mu_stop")

p, s, x_b, s_b = solver.solve()

G = -np.ones_like(solver.grad_p)*solver.G

solution_plot([[solver.x, p],
               [solver.x, s],
               [solver.x, s_b],
               [solver.x, solver.mu_oil_arr],
               [solver.x, solver.grad_p],
               [solver.x, G]],
               titles=("Pressure (p) vs x", "Saturation (s) vs x", "mu_oil", "grad(p)"),
               idx=[[1,1], [1,2], [1,2], [2,1], [2,2], [2,2]],
               nt=solver.nt,
               height=800,
               width=1200)


## Влияние tt на нелинейное течение нефти
при НЕсжимаемом скелете, начальная водонасыщенность = 0

In [ ]:
solver1 = TwoPhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.00002,
                G = 1.4e7,
                mu_water = 8e-4,
                k = 10e-15,
                Q = 0.4/86400,
                m = 0.2,
                beta_r = 0,
                t_0 = 0,
                T = 10*86400,
                nt = 150*5,
                tt = 0.001*864000,
                pow_n = 2,
                alfa_k=1e-9,
                x_0 = 0,
                L = 100,
                nx = 500,
                p_0 = 1e+6,
                s_0 = 0.,
                s_left = 1,
                s_right = 1,
                mu_type = "mu_stop")

p1, s1, x_b1, s_b1 = solver1.solve()

solver2 = TwoPhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.00002,
                G = 1.4e7,
                mu_water = 8e-4,
                k = 10e-15,
                Q = 0.4/86400,
                m = 0.2,
                beta_r = 0,
                t_0 = 0,
                T = 10*86400,
                nt = 150*5,
                tt = 0.2*864000,
                pow_n = 2,
                alfa_k=1e-9,
                x_0 = 0,
                L = 100,
                nx = 500,
                p_0 = 1e+6,
                s_0 = 0.,
                s_left = 1,
                s_right = 1,
                mu_type = "mu_stop")

p2, s2, x_b2, s_b2 = solver2.solve()

solver3 = TwoPhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.00002,
                G = 1.4e7,
                mu_water = 8e-4,
                k = 10e-15,
                Q = 0.4/86400,
                m = 0.2,
                beta_r = 0,
                t_0 = 0,
                T = 10*86400,
                nt = 150*5,
                tt = 0.5*864000,
                pow_n = 2,
                alfa_k=1e-9,
                x_0 = 0,
                L = 100,
                nx = 500,
                p_0 = 1e+6,
                s_0 = 0.,
                s_left = 1,
                s_right = 1,
                mu_type = "mu_stop")

p3, s3, x_b3, s_b3 = solver3.solve()


G = -np.ones_like(solver1.grad_p)*solver1.G


solution_plot([[solver1.x, p1],
               [solver2.x, p2],
               [solver3.x, p3],
               [solver1.x, s1],
               [solver2.x, s2],
               [solver3.x, s3],
               [solver.x, s_b],
               [solver1.x, solver1.mu_oil_arr],
               [solver2.x, solver2.mu_oil_arr],
               [solver3.x, solver3.mu_oil_arr],
               [solver1.x, solver1.grad_p],
               [solver2.x, solver2.grad_p],
               [solver3.x, solver3.grad_p],
               [solver.x, G]],
               titles=("Pressure (p) vs x", "Saturation (s) vs x", "mu_oil", "grad(p)"),
               idx=[[1,1],
                    [1,1],
                    [1,1],
                    [1,2],
                    [1,2],
                    [1,2],
                    [1,2],
                    [2,1],
                    [2,1],
                    [2,1],
                    [2,2],
                    [2,2],
                    [2,2],
                    [2,2]],
               nt=solver.nt,
               height=800,
               width=1200)


## анализ влияния tt нелинейное вытесение
при Несжимаемом скелете и начальной воднонасыщенности = 0.1

In [ ]:
solver1 = TwoPhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.00002,
                G = 1.4e7,
                mu_water = 8e-4,
                k = 10e-15,
                Q = 0.4/86400,
                m = 0.2,
                beta_r = 0,
                t_0 = 0,
                T = 10*86400,
                nt = 150*5,
                tt = 0.001*864000,
                pow_n = 2,
                alfa_k=1e-9,
                x_0 = 0,
                L = 100,
                nx = 500,
                p_0 = 1e+6,
                s_0 = 0.1,
                s_left = 1,
                s_right = 1,
                mu_type = "mu_stop")

p1, s1, x_b1, s_b1 = solver1.solve()

solver2 = TwoPhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.00002,
                G = 1.4e7,
                mu_water = 8e-4,
                k = 10e-15,
                Q = 0.4/86400,
                m = 0.2,
                beta_r = 0,
                t_0 = 0,
                T = 10*86400,
                nt = 150*5,
                tt = 0.2*864000,
                pow_n = 2,
                alfa_k=1e-9,
                x_0 = 0,
                L = 100,
                nx = 500,
                p_0 = 1e+6,
                s_0 = 0.1,
                s_left = 1,
                s_right = 1,
                mu_type = "mu_stop")

p2, s2, x_b2, s_b2 = solver2.solve()

solver3 = TwoPhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.00002,
                G = 1.4e7,
                mu_water = 8e-4,
                k = 10e-15,
                Q = 0.4/86400,
                m = 0.2,
                beta_r = 0,
                t_0 = 0,
                T = 10*86400,
                nt = 150*5,
                tt = 0.5*864000,
                pow_n = 2,
                alfa_k=1e-9,
                x_0 = 0,
                L = 100,
                nx = 200,
                p_0 = 1e+6,
                s_0 = 0.1,
                s_left = 1,
                s_right = 1,
                mu_type = "mu_stop")

p3, s3, x_b3, s_b3 = solver3.solve()


G = -np.ones_like(solver1.grad_p)*solver1.G


solution_plot([[solver1.x, p1],
               [solver2.x, p2],
               [solver3.x, p3],
               [solver1.x, s1],
               [solver2.x, s2],
               [solver3.x, s3],
               [solver1.x, s_b1],
               [solver1.x, solver1.mu_oil_arr],
               [solver2.x, solver2.mu_oil_arr],
               [solver3.x, solver3.mu_oil_arr],
               [solver1.x, solver1.grad_p],
               [solver2.x, solver2.grad_p],
               [solver3.x, solver3.grad_p],
               [solver.x, G]],
               titles=("Pressure (p) vs x", "Saturation (s) vs x", "mu_oil", "grad(p)"),
               idx=[[1,1],
                    [1,1],
                    [1,1],
                    [1,2],
                    [1,2],
                    [1,2],
                    [1,2],
                    [2,1],
                    [2,1],
                    [2,1],
                    [2,2],
                    [2,2],
                    [2,2],
                    [2,2]],
               nt=solver.nt,
               height=800,
               width=1200)


## анализ влияния tt на нелинейное вытеснение
при Сжимаемом скелете, начальной водонасыщенности=0

In [ ]:
solver1 = TwoPhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.00002,
                G = 1.4e7,
                mu_water = 8e-4,
                k = 10e-15,
                Q = 0.4/86400,
                m = 0.2,
                beta_r = 1e-10,
                t_0 = 0,
                T = 10*86400,
                nt = 150*5,
                tt = 0.001*864000,
                pow_n = 2,
                alfa_k=1e-9,
                x_0 = 0,
                L = 100,
                nx = 500,
                p_0 = 1e+6,
                s_0 = 0.,
                s_left = 1,
                s_right = 1,
                mu_type = "mu_stop")

p1, s1, x_b1, s_b1 = solver1.solve()

solver2 = TwoPhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.00002,
                G = 1.4e7,
                mu_water = 8e-4,
                k = 10e-15,
                Q = 0.4/86400,
                m = 0.2,
                beta_r = 1e-10,
                t_0 = 0,
                T = 10*86400,
                nt = 150*5,
                tt = 0.2*864000,
                pow_n = 2,
                alfa_k=1e-9,
                x_0 = 0,
                L = 100,
                nx = 500,
                p_0 = 1e+6,
                s_0 = 0.,
                s_left = 1,
                s_right = 1,
                mu_type = "mu_stop")

p2, s2, x_b2, s_b2 = solver2.solve()

solver3 = TwoPhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.00002,
                G = 1.4e7,
                mu_water = 8e-4,
                k = 10e-15,
                Q = 0.4/86400,
                m = 0.2,
                beta_r = 1e-10,
                t_0 = 0,
                T = 10*86400,
                nt = 150*5,
                tt = 0.5*864000,
                pow_n = 2,
                alfa_k=1e-9,
                x_0 = 0,
                L = 100,
                nx = 500,
                p_0 = 1e+6,
                s_0 = 0.,
                s_left = 1,
                s_right = 1,
                mu_type = "mu_stop")

p3, s3, x_b3, s_b3 = solver3.solve()


G = -np.ones_like(solver1.grad_p)*solver1.G


solution_plot([[solver1.x, p1],
               [solver2.x, p2],
               [solver3.x, p3],
               [solver1.x, s1],
               [solver2.x, s2],
               [solver3.x, s3],
               [solver.x, s_b],
               [solver1.x, solver1.mu_oil_arr],
               [solver2.x, solver2.mu_oil_arr],
               [solver3.x, solver3.mu_oil_arr],
               [solver1.x, solver1.grad_p],
               [solver2.x, solver2.grad_p],
               [solver3.x, solver3.grad_p],
               [solver.x, G]],
               titles=("Pressure (p) vs x", "Saturation (s) vs x", "mu_oil", "grad(p)"),
               idx=[[1,1],
                    [1,1],
                    [1,1],
                    [1,2],
                    [1,2],
                    [1,2],
                    [1,2],
                    [2,1],
                    [2,1],
                    [2,1],
                    [2,2],
                    [2,2],
                    [2,2],
                    [2,2]],
               nt=solver.nt,
               height=800,
               width=1200)


## анализ влияния tt на нелинейное вытеснение
при Сжимаемом скелете, начальной водонасыщенности=0.1

In [ ]:
solver1 = TwoPhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.00002,
                G = 1.4e7,
                mu_water = 8e-4,
                k = 10e-15,
                Q = 0.4/86400,
                m = 0.2,
                beta_r = 1e-10,
                t_0 = 0,
                T = 10*86400,
                nt = 150*5,
                tt = 0.001*864000,
                pow_n = 2,
                alfa_k=1e-9,
                x_0 = 0,
                L = 100,
                nx = 500,
                p_0 = 1e+6,
                s_0 = 0.1,
                s_left = 1,
                s_right = 1,
                mu_type = "mu_stop")

p1, s1, x_b1, s_b1 = solver1.solve()

solver2 = TwoPhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.00002,
                G = 1.4e7,
                mu_water = 8e-4,
                k = 10e-15,
                Q = 0.4/86400,
                m = 0.2,
                beta_r = 1e-10,
                t_0 = 0,
                T = 10*86400,
                nt = 150*5,
                tt = 0.2*864000,
                pow_n = 2,
                alfa_k=1e-9,
                x_0 = 0,
                L = 100,
                nx = 500,
                p_0 = 1e+6,
                s_0 = 0.1,
                s_left = 1,
                s_right = 1,
                mu_type = "mu_stop")

p2, s2, x_b2, s_b2 = solver2.solve()

solver3 = TwoPhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.00002,
                G = 1.4e7,
                mu_water = 8e-4,
                k = 10e-15,
                Q = 0.4/86400,
                m = 0.2,
                beta_r = 1e-10,
                t_0 = 0,
                T = 10*86400,
                nt = 150*5,
                tt = 0.5*864000,
                pow_n = 2,
                alfa_k=1e-9,
                x_0 = 0,
                L = 100,
                nx = 500,
                p_0 = 1e+6,
                s_0 = 0.1,
                s_left = 1,
                s_right = 1,
                mu_type = "mu_stop")

p3, s3, x_b3, s_b3 = solver3.solve()


G = -np.ones_like(solver1.grad_p)*solver1.G


solution_plot([[solver1.x, p1],
               [solver2.x, p2],
               [solver3.x, p3],
               [solver1.x, s1],
               [solver2.x, s2],
               [solver3.x, s3],
               [solver1.x, s_b1],
               [solver1.x, solver1.mu_oil_arr],
               [solver2.x, solver2.mu_oil_arr],
               [solver3.x, solver3.mu_oil_arr],
               [solver1.x, solver1.grad_p],
               [solver2.x, solver2.grad_p],
               [solver3.x, solver3.grad_p],
               [solver1.x, G]],
               titles=("Pressure (p) vs x", "Saturation (s) vs x", "mu_oil", "grad(p)"),
               idx=[[1,1],
                    [1,1],
                    [1,1],
                    [1,2],
                    [1,2],
                    [1,2],
                    [1,2],
                    [2,1],
                    [2,1],
                    [2,1],
                    [2,2],
                    [2,2],
                    [2,2],
                    [2,2]],
               nt=solver.nt,
               height=800,
               width=1200)


## анализ влияния tt на нелинейное течение
при сжимаемом скелете, начальная водонасыщенность=0.1, уменьшенном предельном градиенте

In [ ]:
solver1 = TwoPhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.00002,
                G = 0.4e7,
                mu_water = 8e-4,
                k = 10e-15,
                Q = 0.4/86400,
                m = 0.2,
                beta_r = 1e-10,
                t_0 = 0,
                T = 10*86400,
                nt = 150*5,
                tt = 0.001*864000,
                pow_n = 2,
                alfa_k=1e-9,
                x_0 = 0,
                L = 100,
                nx = 500,
                p_0 = 1e+6,
                s_0 = 0.1,
                s_left = 1,
                s_right = 1,
                mu_type = "mu_stop")

p1, s1, x_b1, s_b1 = solver1.solve()

solver2 = TwoPhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.00002,
                G = 0.4e7,
                mu_water = 8e-4,
                k = 10e-15,
                Q = 0.4/86400,
                m = 0.2,
                beta_r = 1e-10,
                t_0 = 0,
                T = 10*86400,
                nt = 150*5,
                tt = 0.2*864000,
                pow_n = 2,
                alfa_k=1e-9,
                x_0 = 0,
                L = 100,
                nx = 500,
                p_0 = 1e+6,
                s_0 = 0.1,
                s_left = 1,
                s_right = 1,
                mu_type = "mu_stop")

p2, s2, x_b2, s_b2 = solver2.solve()

solver3 = TwoPhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.00002,
                G = 0.4e7,
                mu_water = 8e-4,
                k = 10e-15,
                Q = 0.4/86400,
                m = 0.2,
                beta_r = 1e-10,
                t_0 = 0,
                T = 10*86400,
                nt = 150*5,
                tt = 0.5*864000,
                pow_n = 2,
                alfa_k=1e-9,
                x_0 = 0,
                L = 100,
                nx = 500,
                p_0 = 1e+6,
                s_0 = 0.1,
                s_left = 1,
                s_right = 1,
                mu_type = "mu_stop")

p3, s3, x_b3, s_b3 = solver3.solve()


G = -np.ones_like(solver1.grad_p)*solver1.G


solution_plot([[solver1.x, p1],
               [solver2.x, p2],
               [solver3.x, p3],
               [solver1.x, s1],
               [solver2.x, s2],
               [solver3.x, s3],
               [solver1.x, s_b1],
               [solver1.x, solver1.mu_oil_arr],
               [solver2.x, solver2.mu_oil_arr],
               [solver3.x, solver3.mu_oil_arr],
               [solver1.x, solver1.grad_p],
               [solver2.x, solver2.grad_p],
               [solver3.x, solver3.grad_p],
               [solver1.x, G]],
               titles=("Pressure (p) vs x", "Saturation (s) vs x", "mu_oil", "grad(p)"),
               idx=[[1,1],
                    [1,1],
                    [1,1],
                    [1,2],
                    [1,2],
                    [1,2],
                    [1,2],
                    [2,1],
                    [2,1],
                    [2,1],
                    [2,2],
                    [2,2],
                    [2,2],
                    [2,2]],
               nt=solver.nt,
               height=800,
               width=1200)


*Анализ*
1) Скорость фронта вытеснения (разные предельные градиенты)
2) Скорость увеличения градиента (при разных предельных, и разные верхних)
3) Анализ влияния скорости закачки на добычу
4) Анализ влияния начальной насыщенности
5) Анализ закона вытеснения 
